In [1]:
import sys
import ROOT
import numpy as np
import pandas as pd
import root_pandas
import seaborn as sb
import matplotlib.pyplot as plt
import uproot
import time

from itertools import product

#from root_numpy import root2array

from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc

from bayes_opt import BayesianOptimization
from sklearn.preprocessing import QuantileTransformer
import pickle

Welcome to JupyROOT 6.16/00


/Users/jonamotta/anaconda3/envs/python_root/lib/python2.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.3 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)
Using TensorFlow backend.


In [2]:
#########################################
### PREPARE DFs FOR TRAINING AND TEST ###
#########################################


# some of these features are taken from the LHCb paper and have to be computed and added to the dataframes
features = [
    'm2_miss', 
    'muE_Brf', 
    'q2', 
    'pT_miss',
    'mu_pt'     ,
    'mu_eta'    ,
    'mu_phi'    ,
    #'mu_charge' ,
    'mu1_pt'    ,
    'mu1_eta'   ,
    'mu1_phi'   ,
    #'mu1_charge',
    'mu2_pt'    ,
    'mu2_eta'   ,
    'mu2_phi'   ,
    #'mu2_charge',
]

mu_events = pd.read_csv('mu_events.csv')
tau_events = pd.read_csv('tau_events.csv')

#add the column target to both dataframes
mu_events['target'] = 0
tau_events['target'] = 1

# concatenate the two samples
dataset = pd.concat([mu_events, tau_events],sort=False)

# shuffle and split train/test
train, test = train_test_split(dataset, test_size=0.85, random_state=1986, shuffle=True)

# X and Y on the training sample
X = pd.DataFrame(train, columns=features)
Y = pd.DataFrame(train, columns=['target'])

In [3]:
##################
### PREPROCESS ###
##################


qt = QuantileTransformer(output_distribution='normal', random_state=1986)
qt.fit(X[features])
transformedX = qt.transform(X[features])
pickle.dump( qt, open( 'quantile_tranformation.pck', 'w' ) )

In [4]:
####################################################
### FUNCTION FOR BAYESIAN OPTIMIZATION OF THE NN ###
####################################################


def BO_function(features,transformedX,Y,test,pbounds,init_points,n_iter):
    start = time.time()
    def NN_function(n_layers,units_perlayer,batch_size):
    
        #optimizer_fcts = ['nadam', 'adamax', 'adam', 'adadelta', 'adagrad', 'rmsprop', 'sgd']
        #activation_fcts = ['softmax', 'elu', 'selu', 'relu', 'softplus', 'softsign', 'tanh', 'sigmoid', 
        #                   'hard_sigmoid', 'exponential']
        #metrics_fcts = ['binary_accuracy', 'categorical_accuracy', 'sparse_categorical_accuracy', 
        #                'top_k_categorical_accuracy', 'sparse_top_k_categorical_accuracy']
        #loss_fcts = ['mean_squared_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
        #            'squared_hinge', 'hinge', 'categorical_hinge', 'logcosh', 'categorical_crossentropy',
        #            'sparse_categorical_crossentropy', 'binary_crossentropy', 'kullback_leibler_divergence',
        #            'poisson', 'cosine_proximity']
        
        # I want units_perlayer to be a multiple of 2 -> I always take le lower multiple of 2 starting from the
        # float that the algorith is giving me
        if units_perlayer%2 < 1:
            units_perlayer = int(units_perlayer)
        else:
            units_perlayer = int(units_perlayer-1)

        # define the model
        model = Sequential()
        for i in range(int(n_layers)):
            model.add(Dense(units_perlayer, input_dim=len(features),activation='relu'))
        model.add(Dense(1,activation='sigmoid'))

        # compile the model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # fit the model
        model.fit(transformedX, Y, epochs=20, batch_size=int(batch_size), validation_split=0.5,verbose=0)

        # evaluate the model
        scores = model.evaluate(transformedX, Y,verbose=0)

        # calculate predictions on the test sample
        x = pd.DataFrame(test, columns=features)
        qt = pickle.load(open( 'quantile_tranformation.pck', 'r' ))
        transformedx = qt.transform(x[features])
        y = model.predict(transformedx)
    
        # create this random in order to avoid repetitions in the insertion of the score of the NN (nnS)
        k = np.random.normal(100,20)
        
        # add the score to the test sample dataframe
        test.insert(len(test.columns), 'nnS'+str(k), y)

        # let sklearn do the heavy lifting and compute the ROC curves for you
        fpr, tpr, wps = roc_curve(test.target, test['nnS'+str(k)])

        # compute the auc
        auroc = auc(fpr, tpr)

        # compute Gini index
        gini_index = (auroc-0.5)*2
        
        return np.log((auroc*gini_index*scores[1])**4)

    optimizer = BayesianOptimization(
        f = NN_function,
        pbounds = pbounds,
    )
    
    # optimize
    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
        alpha=1e-3,
    )
    
    print optimizer.max
    
    end = time.time()
    print 'Running time of the Bayesian Optimization = %.1f'%(end - start)

In [5]:
#########################################
### OPTIMIZE THE PARAMETERS OF THE NN ###
#########################################


# bounded region of parameter space
pbounds = {'n_layers': (1, 10), 'units_perlayer': (16, 128), 'batch_size': (5, 5000)}
# , 'optimizer': (0,7), 'activation': (0,10), 'loss': (0,13)




# call the function that creates the optimizer
BO_function(features,transformedX,Y,test,pbounds,40,60)

|   iter    |  target   | batch_... | n_layers  | units_... |
-------------------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
|  1        | -1.288    |  1.773e+0 |  5.785    |  54.05    |
|  2        | -1.533    |  3.994e+0 |  1.133    |  111.6    |
|  3        | -1.336    |  660.4    |  5.347    |  118.3    |
|  4        | -1.306    |  479.2    |  1.187    |  27.49    |
|  5        | -1.379    |  2.408e+0 |  3.108    |  20.31    |
|  6        | -1.283    |  815.8    |  3.074    |  36.83    |
|  7        | -1.288    |  1.046e+0 |  2.52     |  60.55    |
|  8        | -1.288    |  2.334e+0 |  8.332    |  64.95    |
|  9        | -1.306    |  2.335e+0 |  8.948    |  88.37    |
|  10       | -1.284    |  3.691e+0 |  7.731    |  109.2    |
|  11       | -1.353    |  196.4    |  4.859    |  76.37    |
|  12       | -1.409    |  411.0    |  9.461    |  125.6    |
|  13       

|  74       | -1.351    |  679.3    |  1.0      |  16.0     |
|  75       | -1.288    |  3.539e+0 |  10.0     |  128.0    |
|  76       | -1.368    |  2.526e+0 |  1.0      |  128.0    |
|  77       | -1.442    |  2.646e+0 |  10.0     |  16.0     |
|  78       | -1.292    |  2.151e+0 |  10.0     |  128.0    |
|  79       | -1.283    |  4.415e+0 |  10.0     |  128.0    |
|  80       | -2.265    |  4.454e+0 |  10.0     |  16.0     |
|  81       | -2.013    |  1.901e+0 |  1.0      |  16.0     |
|  82       | -2.431    |  3.047e+0 |  1.0      |  16.0     |
|  83       | -1.325    |  2.853e+0 |  10.0     |  128.0    |
|  84       | -1.498    |  4.145e+0 |  1.0      |  128.0    |
|  85       | -1.481    |  3.587e+0 |  10.0     |  16.0     |
|  86       | -1.368    |  2.186e+0 |  10.0     |  16.0     |
|  87       | -1.557    |  3.359e+0 |  10.0     |  16.0     |
|  88       | -1.287    |  4.591e+0 |  10.0     |  128.0    |
|  89       | -3.468    |  4.768e+0 |  1.0      |  16.0     |
|  90   

2019-07-16 21:59:17.218068: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
